In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import LlamaTokenizer, LlamaModel

/home/geffte-caetano/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = pd.read_excel('/home/geffte-caetano/Documents/GitHub/ecos-da-rota/datasets/dataset deteccao.xlsx')

In [3]:
dataset.describe()

,text,label
count,9450,9450
unique,9424,36
top,A umidade relativa do ar deve atingir níveis c...,Arboviroses
freq,2,1586


In [4]:
dataset['label'] = dataset['label'].astype('category').cat.codes

In [5]:
dataset.head()

,text,label
0,A estiagem prolongada em 2021 fez com diretore...,34
1,"Nos últimos 20 dias, 40 pessoas morreram em Ca...",8
2,Os meses de agosto e setembro prometem ser dec...,8
3,"Uma experiência ""revolucionária"" com mosquitos...",1
4,"Novo decreto publicado nesta sexta-feira (20),...",8


In [6]:
samples_per_label = 2

df_small = dataset.groupby('label').filter(lambda x: len(x) >= samples_per_label)
df_small = df_small.groupby('label').apply(lambda x: x.sample(samples_per_label)).reset_index(drop=True)


/tmp/ipykernel_29233/1879449949.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_small = df_small.groupby('label').apply(lambda x: x.sample(samples_per_label)).reset_index(drop=True)


In [7]:
df_small

,text,label
0,O diretor-geral da Organização Mundial da Saúd...,0
1,A Secretaria de Saúde do Distrito Federal crio...,0
2,Dos 144 municípios considerados infestados pel...,1
3,"O inverno já chegou e a chuva ficou para trás,...",1
4,A Associação Nacional do Transporte de Cargas ...,2
...,...,...
65,A vítima de estupro na Cadeia Pública José Fre...,33
66,A Ministra de Estado do Meio Ambiente e Mudanç...,34
67,"As ondas de calor, como as que ocorreram em se...",34
68,PARIS (Reuters) – O Gabão relatou um surto de ...,35


In [8]:
df = df_small

In [9]:
# Carregar o tokenizer e o modelo LLaMA
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b")
model = LlamaModel.from_pretrained("meta-llama/Llama-2-7b")

OSError: Can't load tokenizer for 'meta-llama/Llama-2-7b'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'meta-llama/Llama-2-7b' is the correct path to a directory containing all relevant files for a LlamaTokenizer tokenizer.

In [ ]:
def generating_embedding(text, tokenizer, model):
    # Tokenizar o texto
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Passar os tokens pelo modelo
    with torch.no_grad():
        outputs = model(**inputs)

    # Pegue o embedding da última camada oculta
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

    return embeddings


In [ ]:
checkpoint = 'llama3'
text_column_name = 'text'

df['embedding'] = df.apply(lambda row: generating_embedding(row, text_column_name, checkpoint), axis=1)

dataset.head()

ConnectError: [Errno 111] Connection refused